In [214]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score

In [113]:
dataset_df = pd.read_csv('./dataset/iris_train.csv')

In [114]:
print(dataset_df.shape)
dataset_df.head()

(75, 6)


,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,setosa,4.4,1.4,2.9,0.2
1,1,versicolor,6.4,4.5,3.2,1.5
2,2,virginica,6.2,4.8,2.8,1.8
3,3,virginica,7.2,6.1,3.6,2.5
4,4,setosa,4.9,1.4,3.0,0.2


In [115]:
dataset_df['species'] = dataset_df['species'].map({'setosa':0, 'versicolor': 1, 'virginica': 2})

In [116]:
dataset_df

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,0,4.4,1.4,2.9,0.2
1,1,1,6.4,4.5,3.2,1.5
2,2,2,6.2,4.8,2.8,1.8
3,3,2,7.2,6.1,3.6,2.5
4,4,0,4.9,1.4,3.0,0.2
...,...,...,...,...,...,...
70,70,1,6.5,4.6,2.8,1.5
71,71,1,5.6,3.6,2.9,1.3
72,72,1,6.2,4.5,2.2,1.5
73,73,1,4.9,3.3,2.4,1.0


In [139]:
X, Y1, Y2 = dataset_df.iloc[:,1:4] , dataset_df['sepal width (cm)'], dataset_df['petal width (cm)']

In [165]:
model_count = 5

model1 = []
model2 = []

In [166]:
kf = KFold(n_splits=model_count)
kf.get_n_splits(X)

for train_index, test_index in kf.split(X):

    x_train, x_test = X.values[train_index], X.values[test_index]
    y1_train, y1_test = Y1.values[train_index], Y1.values[test_index]
    y2_train, y2_test = Y2.values[train_index], Y2.values[test_index]

    
    created_model1 = MLPRegressor(random_state=1, max_iter=300).fit(x_train, y1_train)
    created_model2 = MLPRegressor(random_state=1, max_iter=300).fit(x_train, y2_train)
    
    model1.append(created_model1)
    model2.append(created_model2)


In [187]:
result_df = pd.read_csv('./dataset/iris_test.csv')
result_df['species'] = result_df['species'].map({'setosa':0, 'versicolor': 1, 'virginica': 2})
result_df.head()

x_result = result_df.iloc[:,1:4]
x_result

,species,sepal length (cm),petal length (cm)
0,0,5.4,1.7
1,0,5.7,1.5
2,0,5.3,1.5
3,0,5.1,1.9
4,2,6.0,4.8
...,...,...,...
70,1,5.8,4.0
71,0,4.4,1.3
72,2,6.3,5.6
73,1,5.1,3.0


In [209]:
result_data1 = np.zeros_like(model1[0].predict(x_result))
result_data2 = np.zeros_like(model2[0].predict(x_result))

for i in range(model_count):

    result_data1 += model1[i].predict(x_result)
    result_data2 += model2[i].predict(x_result)

result_data1 /= model_count
result_data2 /= model_count

In [212]:
submission = pd.read_csv('./dataset/sample_submission.csv')
submission['sepal width (cm)'] = result_data1
submission['petal width (cm)'] = result_data2

In [215]:
submission.to_csv("result.csv", index=False)